### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import sys

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv('../my_keys.env')

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")
print(tmdb_api_key)

839a6971a94fccf9f0345fe572c6a6a9


### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}fq={filter_query}&api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}&fl={field_list}&sort={sort}"


In [4]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    page_url = f"{query_url}&page={page}"
    # create query with a page number
    # API results show 10 articles at a time
    # # Make a "GET" request and retrieve the JSON
    reviews = requests.get(page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    if page > 0:
        time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        docs = reviews['response']['docs']
        # loop through the reviews["response"]["docs"] and append each review to the list
        for doc in docs:
            reviews_list.append(doc)
        # Print the page that was just retrieved
        print(f"Page {page+1} successfully retrieved!")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No results found on page {page+1}.")
        break

Page 1 successfully retrieved!
Page 2 successfully retrieved!
Page 3 successfully retrieved!
Page 4 successfully retrieved!
Page 5 successfully retrieved!
Page 6 successfully retrieved!
Page 7 successfully retrieved!
Page 8 successfully retrieved!
Page 9 successfully retrieved!
Page 10 successfully retrieved!
Page 11 successfully retrieved!
Page 12 successfully retrieved!
Page 13 successfully retrieved!
Page 14 successfully retrieved!
Page 15 successfully retrieved!
Page 16 successfully retrieved!
Page 17 successfully retrieved!
Page 18 successfully retrieved!
Page 19 successfully retrieved!
Page 20 successfully retrieved!


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5],indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [7]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
reviews_df['title']=reviews_df['headline.main'].apply(lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")])
reviews_df.head(1)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries


In [8]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df['keywords'] = reviews_df['keywords'].apply(extract_keywords)
reviews_df.head(2)

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo


In [9]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_df['title'].to_list()

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
#'https://api.themoviedb.org/3/search/movie?query=Jack+Reacher&api_key=API_KEY'

url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [11]:
# #Search format
# response = requests.get(f"{url}{titles[0]}&api_key={tmdb_api_key}").json()
# print(json.dumps(response,indent=4))
# #detail_format
# detail_response = requests.get(f"https://api.themoviedb.org/3/movie/743040?api_key={tmdb_api_key}").json()
# print(json.dumps(detail_response,indent=4))

In [12]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1

# Loop through the titles
    # Check if we need to sleep before making a request
    # Add 1 to the request counter
    # Perform a "GET" request for The Movie Database
for title in titles:
    if request_counter % 50 == 0:
        print(f"Sleeping on loop {request_counter}."+" "*50,flush=True)
        time.sleep(1)
        
    response = requests.get(f"{url}{title}&api_key={tmdb_api_key}").json()

    request_counter += 1
    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try: 
        # Get movie id
        # Make a request for a the full movie details
        # Execute "GET" request with url        
        # Extract the genre names into a list
        # Extract the spoken_languages' English name into a list
        # Extract the production_countries' name into a list
        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list     
        # Print out the title that was found
        movie_id = response['results'][0]['id']

        movie_query = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
        movie_details = requests.get(movie_query).json()

        genre_names = []
        for genre in movie_details['genres']:
            genre_names.append(genre['name'])
        
        languages = []
        for language in movie_details['spoken_languages']:
            languages.append(language['english_name'])
        
        countries = []
        for country in movie_details['production_countries']:
            countries.append(country['name'])
        
        movie_dict = {'title':movie_details['title'],
                        'original_title':movie_details['original_title'],
                        'budget':movie_details['budget'],
                        'original_language':movie_details['original_language'],
                        'homepage':movie_details['homepage'],
                        'overview':movie_details['overview'],
                        'popularity':movie_details['popularity'],
                        'runtime':movie_details['runtime'],
                        'revenue':movie_details['revenue'],
                        'release_date':movie_details['release_date'],
                        'vote_average':movie_details['vote_average'],
                        'vote_count':movie_details['vote_count'],
                        'genres':genre_names,
                        'spoken_languages':languages,
                        'production_countries':countries}
        tmdb_movies_list.append(movie_dict)

        print(f"Loop {request_counter}: Found {title}."+' '*50, end='\r', flush=True)
        
    except Exception as e:
        print(f"In Loop {request_counter}, {title[:30]} wasn't found. Resulting in the following error: {e}. The failed json was: {response}")





In Loop 3, What’s Love Got to Do With It? wasn't found. Resulting in the following error: list index out of range. The failed json was: {'page': 1, 'results': [], 'total_pages': 1, 'total_results': 0}
In Loop 20, Bones and All wasn't found. Resulting in the following error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)). The failed json was: {'page': 1, 'results': [{'adult': False, 'backdrop_path': '/8sPxa4sdRvjgRG3GgkO8RQxUR9P.jpg', 'genre_ids': [18, 27, 10749], 'id': 791177, 'original_language': 'it', 'original_title': 'Bones and All', 'overview': 'Abandoned by her father, a young woman embarks on a thousand-mile odyssey through the backroads of America where she meets a disenfranchised drifter. But despite their best efforts, all roads lead back to their terrifying pasts and to a final stand that will determine whether their love can survive their otherness.', 'popularity': 25.83, 'poster_path'

In [13]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 0.777,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 4.0,
        "vote_count": 2,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
        ]
 

In [14]:
# Convert the results to a DataFrame
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",0.777,102,0,2021-10-07,4.000,2,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",9.805,96,0,2022-06-11,7.300,38,"[Drama, Romance]","[French, English]",[Canada]
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",20.425,96,0,2023-04-21,6.392,139,"[Romance, Comedy, Adventure]",[English],[United States of America]
3,Other People's Children,Les Enfants des autres,0,fr,https://www.wildbunch.biz/movie/other-peoples-...,"Rachel loves her life, her students, her frien...",8.753,104,84178,2022-09-21,6.900,157,"[Drama, Comedy]","[French, English]",[France]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,10.080,100,37820,2023-04-07,6.813,48,"[Romance, Drama, Comedy]",[English],"[Germany, United States of America]"


### Merge and Clean the Data for Export

In [15]:
# Merge the New York Times reviews and TMDB DataFrames on title
nyt_tmdb_merged_df = pd.merge(tmdb_df,reviews_df, on='title')

In [16]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres','spoken_languages','production_countries']

# Create a list of characters to remove
characters_to_remove = ['[',']',"'"]

# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    nyt_tmdb_merged_df[column] = nyt_tmdb_merged_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:
        nyt_tmdb_merged_df[column] = nyt_tmdb_merged_df[column].str.replace(character,'')

# Display the fixed DataFrame
#The full result had my columns of interest in the truncated section, so the following subset was used 
nyt_tmdb_merged_df[columns_to_fix].head()

,genres,spoken_languages,production_countries
0,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,"Drama, Romance","French, English",Canada
2,"Romance, Drama, Comedy",English,"Germany, United States of America"
3,Documentary,English,United States of America
4,Drama,English,United States of America


In [17]:
# Drop "byline.person" column
nyt_tmdb_merged_df.drop(columns='byline.person', inplace=True)

In [18]:
# Delete duplicate rows and reset index
nyt_tmdb_merged_df.drop_duplicates(inplace=True)
nyt_tmdb_merged_df.reset_index(drop=True)

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,...,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",0.777,102,0,2021-10-07,...,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",9.805,96,0,2022-06-11,...,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
2,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,10.080,100,37820,2023-04-07,...,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,None
3,The Lost Weekend: A Love Story,The Lost Weekend: A Love Story,0,en,https://thelostweekendmovie.com/,May Pang lovingly recounts her life in rock & ...,7.207,95,0,2023-04-13,...,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,None
4,A Thousand and One,A Thousand and One,0,en,https://www.focusfeatures.com/a-thousand-and-one/,Struggling but unapologetically living on her ...,27.565,116,0,2023-03-31,...,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,In Search of Fellini,In Search of Fellini,0,en,,"Lucy, a small-town girl from Ohio, discovers t...",6.905,93,0,2017-09-15,...,272,Review: Love of Il Maestro Drives ‘In Search o...,None,None,In Search of Fellini,None,None,None,By Monica Castillo,None
118,The Last Face,The Last Face,0,en,,"Miguel, a heroic Spanish doctor, puts himself ...",12.382,130,0,2017-01-11,...,280,Review: Aid Workers in Love and War in Sean Pe...,None,None,The Last Face,None,None,None,By Glenn Kenny,None
119,Lost in Paris,Paris pieds nus,0,fr,,Fiona visits Paris for the first time to assis...,10.584,83,0,2017-01-14,...,254,Review: Finding Love (and Slapstick) While ‘Lo...,None,None,Lost in Paris,None,None,None,By Ben Kenigsberg,None
120,The Other Half,The Other Half,0,en,http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,5.498,103,0,2016-12-02,...,251,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,None


In [19]:
# Export data to CSV without the index
nyt_tmdb_merged_df.to_csv('./output/final_collected_data.csv',index=False)